In [3]:
!pip install --upgrade pip
!pip install pickle5
!pip install --upgrade pandas
!pip install numpy
!pip install scikit-learn==0.24.0
#!pip install pandas==0.24.1
!pip install wandb -qq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
floyd-cli 0.11.17 requires click<7,>=6.7, but you have click 7.1.2 which is incompatible.


In [1]:
#!pip install --upgrade pandas
import pandas as pd 
import pickle5 as pickle
from tqdm import tqdm
from time import time
import numpy as np
import sklearn


In [2]:
#Transforming outliers imputations
def median_imputation(df, field_name):
    q1 = df[field_name].quantile(0.25)
    q1 = df[field_name].quantile(0.25)
    q3 = df[field_name].quantile(0.75)
    iqr = q3-q1
    Lower_tail = q1 - 1.5 * iqr
    Upper_tail = q3 + 1.5 * iqr
    med = np.median(df[field_name])
    for i in df[field_name]:
        if i > Upper_tail or i < Lower_tail:
            df[field_name] = df[field_name].replace(i, med)
    return df

In [3]:
with open("data_ML2021_01_07_15_36_52.pkl", "rb") as fh:
  data = pickle.load(fh)


In [4]:
import pandas as pd

data = pd.read_pickle(r'data_ML2021_01_07_15_36_52.pkl')

In [5]:
data.columns

Index(['opis', 'cena', 'data dodania', 'lokalizacja', 'Na_sprzedaż_przez',
       'Rodzaj_nieruchomosci', 'Liczba_pokoi', 'Liczba_łazienek',
       'Wielkość (m2)', 'Parking', 'mieszkanie_url', 'dzielnica', 'miasto',
       'cena_za_metr', 'atrakcyjnosc', 'poziom_atrakcyjnosci',
       'ekologia_ranking', 'bezpieczenstwo_ranking', 'opis_clean',
       'opis_cleanv2', 'opis_clean_nlp', 'opis_cleanv2_nlp'],
      dtype='object')

In [6]:
### ML without opis with outliers
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
with open("data_ML2021_01_07_15_36_52.pkl", "rb") as fh:
  data = pickle.load(fh)

median_imputation(data,'cena_za_metr')
median_imputation(data,'Wielkość (m2)')
testdata = pd.get_dummies(data,columns=['Na_sprzedaż_przez', 'Rodzaj_nieruchomosci','Liczba_pokoi', 'Liczba_łazienek', 'Parking', 'dzielnica','miasto','poziom_atrakcyjnosci'])
y = testdata['cena_za_metr'].values
X = testdata.drop(['cena_za_metr','cena','mieszkanie_url', 'lokalizacja','opis','opis_clean','opis_cleanv2','opis_clean_nlp','opis_cleanv2_nlp','data dodania'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=17)
scores = cross_val_score(LinearRegression(), X_train, y_train, cv=5,n_jobs=-1,verbose=15)
print(list(scores))
print()
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[CV]  ................................................................
[CV]  ................................................................
[CV] ....................... , score=0.4364543261896323, total=   0.1s
[CV] ....................... , score=0.4467320347677121, total=   0.1s
[CV]  ................................................................
[CV]  ................................................................


/usr/local/lib/python3.6/site-packages/sklearn/externals/joblib/numpy_pickle.py:93: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1757s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.2s


[CV] ....................... , score=0.4338428205592179, total=   0.1s
[CV]  ................................................................
[CV] ....................... , score=0.4433085604955055, total=   0.2s
[CV] ...................... , score=0.45537371045307357, total=   0.1s


[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    0.4s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished


[0.4364543261896323, 0.4467320347677121, 0.4433085604955055, 0.4338428205592179, 0.45537371045307357]

Accuracy: 0.44 (+/- 0.02)


In [7]:
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer
scoring='f1_macro'

with open("data_ML2021_01_07_15_36_52.pkl", "rb") as fh:
  data = pickle.load(fh)

median_imputation(data,'cena_za_metr')
median_imputation(data,'Wielkość (m2)')
testdata = pd.get_dummies(data,columns=['Na_sprzedaż_przez', 'Rodzaj_nieruchomosci','Liczba_pokoi', 'Liczba_łazienek', 'Parking', 'dzielnica','miasto','poziom_atrakcyjnosci'])
y = testdata['cena_za_metr'].values
X = testdata.drop(['cena_za_metr','cena','mieszkanie_url', 'lokalizacja','opis','opis_clean','opis_cleanv2','opis_clean_nlp','opis_cleanv2_nlp','data dodania'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=17)
scores = cross_val_score(LinearRegression(), X_train, y_train, scoring=make_scorer(mean_squared_error), cv=5, n_jobs=-1,verbose=15)
print(list(scores))
print()
print("Mean square error: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[CV]  ................................................................
[CV]  ................................................................
[CV] ....................... , score=3555631.7581710448, total=   0.1s
[CV] ....................... , score=3594786.8792736945, total=   0.1s
[CV]  ................................................................
[CV]  ................................................................


/usr/local/lib/python3.6/site-packages/sklearn/externals/joblib/numpy_pickle.py:93: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1770s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.2s


[CV] ....................... , score=3790972.2953676092, total=   0.2s
[CV] ....................... , score=3648859.6293048416, total=   0.2s
[CV]  ................................................................
[CV] ........................ , score=3516623.601419095, total=   0.1s


[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    0.4s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished


[3555631.7581710448, 3594786.8792736945, 3648859.6293048416, 3790972.2953676092, 3516623.601419095]

Mean square error: 3621374.83 (+/- 190831.04)


In [9]:

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


pipeline = Pipeline([
                ('selector', ItemSelector(key='opis_cleanv2_nlp')),
                ('tfidf', TfidfVectorizer(smooth_idf=True)),
                ('best', TruncatedSVD(n_components=250)),
                ('linear', LinearRegression())
            ])

with open("data_ML2021_01_07_15_36_52.pkl", "rb") as fh:
  data = pickle.load(fh)
median_imputation(data,'cena_za_metr')
median_imputation(data,'Wielkość (m2)')
testdata = pd.get_dummies(data,columns=['Na_sprzedaż_przez', 'Rodzaj_nieruchomosci','Liczba_pokoi', 'Liczba_łazienek', 'Parking', 'dzielnica','miasto','poziom_atrakcyjnosci'])
y = testdata['cena_za_metr'].values
X = testdata.drop(['cena_za_metr','cena','mieszkanie_url', 'lokalizacja','opis','opis_clean','opis_cleanv2','opis_clean_nlp','data dodania'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
scores = cross_val_score(pipeline, X_train, y_train, cv=3,n_jobs=-1,verbose=15)
print(list(scores))
print()
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[CV]  ................................................................


/usr/local/lib/python3.6/site-packages/sklearn/externals/joblib/numpy_pickle.py:93: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


[CV]  ................................................................
[CV] ...................... , score=0.40087248102319206, total=  15.1s
[CV] ...................... , score=0.39726006470809405, total=  15.3s
[CV]  ................................................................


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   15.5s


[CV] ....................... , score=0.3987065341340176, total=   8.2s
[0.39726006470809405, 0.40087248102319206, 0.3987065341340176]

Accuracy: 0.40 (+/- 0.00)


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   23.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   23.9s finished


In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


pipeline = Pipeline([
                ('selector', ItemSelector(key='opis_cleanv2_nlp')),
                ('tfidf', TfidfVectorizer()),
                ('best', TruncatedSVD(n_components=250)),
                ('linear', SVR(kernel='linear', C=1000, gamma='auto'))
            ])

with open("data_ML2021_01_07_15_36_52.pkl", "rb") as fh:
  data = pickle.load(fh)
median_imputation(data,'cena_za_metr')
median_imputation(data,'Wielkość (m2)')
testdata = pd.get_dummies(data,columns=['Na_sprzedaż_przez', 'Rodzaj_nieruchomosci','Liczba_pokoi', 'Liczba_łazienek', 'Parking', 'dzielnica','miasto','poziom_atrakcyjnosci'])
y = testdata['cena_za_metr'].values
X = testdata.drop(['cena_za_metr','cena','mieszkanie_url', 'lokalizacja','opis','opis_clean','opis_cleanv2','opis_clean_nlp','data dodania'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
scores = cross_val_score(pipeline, X_train, y_train, cv=5,verbose=15,n_jobs=-1)
print(list(scores))
print()
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[CV]  ................................................................


/usr/local/lib/python3.6/site-packages/sklearn/externals/joblib/numpy_pickle.py:93: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


[CV]  ................................................................
[CV] ....................... , score=0.3626722577411884, total= 7.8min
[CV]  ................................................................


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  7.8min


[CV] ....................... , score=0.3622469109800719, total= 8.3min
[CV]  ................................................................


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  8.3min


[CV] ........................ , score=0.378221852992373, total= 7.9min
[CV]  ................................................................


[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed: 15.7min remaining: 10.5min


[CV] ....................... , score=0.3594149666522408, total= 8.4min
[CV] ....................... , score=0.3370261830150276, total= 6.6min
[0.3626722577411884, 0.3622469109800719, 0.378221852992373, 0.3594149666522408, 0.3370261830150276]

Accuracy: 0.36 (+/- 0.03)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 22.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 22.3min finished


In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
#sklearn.svm.LinearSVR
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


pipeline = Pipeline([
                ('selector', ItemSelector(key='opis_cleanv2_nlp')),
                ('tfidf', TfidfVectorizer()),
                ('best', TruncatedSVD(n_components=350)),
                ('linear', SVR(kernel='linear', C=1000, gamma='auto'))
            ])

with open("data_ML2021_01_07_15_36_52.pkl", "rb") as fh:
  data = pickle.load(fh)
median_imputation(data,'cena_za_metr')
median_imputation(data,'Wielkość (m2)')
testdata = pd.get_dummies(data,columns=['Na_sprzedaż_przez', 'Rodzaj_nieruchomosci','Liczba_pokoi', 'Liczba_łazienek', 'Parking', 'dzielnica','miasto','poziom_atrakcyjnosci'])
y = testdata['cena_za_metr'].values
X = testdata.drop(['cena_za_metr','cena','mieszkanie_url', 'lokalizacja','opis','opis_clean','opis_cleanv2','opis_clean_nlp','data dodania'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
scores = cross_val_score(pipeline, X_train, y_train, cv=5,n_jobs=-1,verbose=15)
print(list(scores))
print()
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[CV]  ................................................................


/usr/local/lib/python3.6/site-packages/sklearn/externals/joblib/numpy_pickle.py:93: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


[CV]  ................................................................
[CV] ....................... , score=0.3823134329707503, total=10.9min
[CV]  ................................................................


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 10.9min


[CV] ....................... , score=0.3828936858474194, total=11.5min
[CV]  ................................................................


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 11.5min


[CV] ........................ , score=0.405599255960666, total=11.0min
[CV]  ................................................................


[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed: 21.9min remaining: 14.6min


[CV] ....................... , score=0.3915199671608081, total=10.9min
[CV] ....................... , score=0.3666822775615214, total= 9.4min
[0.3828936858474194, 0.3823134329707503, 0.405599255960666, 0.3915199671608081, 0.3666822775615214]

Accuracy: 0.39 (+/- 0.03)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 31.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 31.3min finished


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


pipeline = Pipeline([
                ('selector', ItemSelector(key='opis_cleanv2_nlp')),
                ('tfidf', TfidfVectorizer()),
                ('best', TruncatedSVD(n_components=350)),
                ('linear', LinearSVR(C=1000))
            ])

with open("data_ML2021_01_07_15_36_52.pkl", "rb") as fh:
  data = pickle.load(fh)
median_imputation(data,'cena_za_metr')
median_imputation(data,'Wielkość (m2)')
testdata = pd.get_dummies(data,columns=['Na_sprzedaż_przez', 'Rodzaj_nieruchomosci','Liczba_pokoi', 'Liczba_łazienek', 'Parking', 'dzielnica','miasto','poziom_atrakcyjnosci'])
y = testdata['cena_za_metr'].values
X = testdata.drop(['cena_za_metr','cena','mieszkanie_url', 'lokalizacja','opis','opis_clean','opis_cleanv2','opis_clean_nlp','data dodania'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
scores = cross_val_score(pipeline, X_train, y_train, cv=5,n_jobs=-1,verbose=15)
print(list(scores))
print()
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from time import time
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

parameters = parameters = {
    'best__n_components': (350,500,750,1000),
    'linear__C': (100,1000,10000)
}

pipeline = Pipeline([
                ('selector', ItemSelector(key='opis_cleanv2_nlp')),
                ('tfidf', TfidfVectorizer()),
                ('best', TruncatedSVD()),
                ('linear', LinearSVR())
            ])

grid_search = GridSearchCV(pipeline, parameters, cv=3,n_jobs=-1,verbose=15)


with open("data_ML2021_01_07_15_36_52.pkl", "rb") as fh:
  data = pickle.load(fh)
median_imputation(data,'cena_za_metr')
median_imputation(data,'Wielkość (m2)')
testdata = pd.get_dummies(data,columns=['Na_sprzedaż_przez', 'Rodzaj_nieruchomosci','Liczba_pokoi', 'Liczba_łazienek', 'Parking', 'dzielnica','miasto','poziom_atrakcyjnosci'])
y = testdata['cena_za_metr'].values
X = testdata.drop(['cena_za_metr','cena','mieszkanie_url', 'lokalizacja','opis','opis_clean','opis_cleanv2','opis_clean_nlp','data dodania'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=17)

t0 = time()
grid_search.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from time import time
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

parameters = parameters = {
    'best__n_components': (750,1000),
    'svr__C': (100,1000),
    'svr__kernel':('linear', 'rbf')
}

pipeline = Pipeline([
                ('selector', ItemSelector(key='opis_cleanv2_nlp')),
                ('tfidf', TfidfVectorizer()),
                ('best', TruncatedSVD()),
                ('svr', SVR())
            ])

grid_search = GridSearchCV(pipeline, parameters, verbose=30, cv=3,n_jobs=-1)


with open("data_ML2021_01_07_15_36_52.pkl", "rb") as fh:
  data = pickle.load(fh)
median_imputation(data,'cena_za_metr')
median_imputation(data,'Wielkość (m2)')
testdata = pd.get_dummies(data,columns=['Na_sprzedaż_przez', 'Rodzaj_nieruchomosci','Liczba_pokoi', 'Liczba_łazienek', 'Parking', 'dzielnica','miasto','poziom_atrakcyjnosci'])
y = testdata['cena_za_metr'].values
X = testdata.drop(['cena_za_metr','cena','mieszkanie_url', 'lokalizacja','opis','opis_clean','opis_cleanv2','opis_clean_nlp','data dodania'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=17)

t0 = time()
grid_search.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] best__n_components=750, svr__C=100, svr__kernel=linear ..........
[CV] best__n_components=750, svr__C=100, svr__kernel=linear ..........
[CV]  best__n_components=750, svr__C=100, svr__kernel=linear, score=0.3802497743777943, total=16.6min
[CV] best__n_components=750, svr__C=100, svr__kernel=linear ..........


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 26.9min


[CV]  best__n_components=750, svr__C=100, svr__kernel=linear, score=0.3808465125011462, total=16.7min
[CV] best__n_components=750, svr__C=100, svr__kernel=rbf .............


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 27.1min


In [5]:

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from warnings import simplefilter
simplefilter(action='ignore', category=DeprecationWarning)
from sklearn.model_selection import GridSearchCV


param_grid = dict( scale=['passthrough', StandardScaler(), Normalizer()]
)
                  
print(param_grid)

with open("data_ML2021_01_07_15_36_52.pkl", "rb") as fh:
  data = pickle.load(fh)
median_imputation(data,'cena_za_metr')
median_imputation(data,'Wielkość (m2)')
testdata = pd.get_dummies(data,columns=['Na_sprzedaż_przez', 'Rodzaj_nieruchomosci','Liczba_pokoi', 'Liczba_łazienek', 'Parking', 'dzielnica','miasto','poziom_atrakcyjnosci'])
y = testdata['cena_za_metr'].values
X = testdata.drop(['cena_za_metr','cena','mieszkanie_url', 'lokalizacja','opis','opis_clean','opis_cleanv2','opis_clean_nlp','data dodania','opis_cleanv2_nlp'], axis=1)

pipe = Pipeline([
    ('scale',  'passthrough'),
    ('regression', Ridge())
])

grid_search = GridSearchCV(pipe, param_grid, verbose=30, cv=3,n_jobs=-1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=17)

t0 = time()
grid_search.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best parameters set:")
print(grid_search.best_estimator_)
print(grid_search.best_score_)

{'scale': ['passthrough', StandardScaler(copy=True, with_mean=True, with_std=True), Normalizer(copy=True, norm='l2')]}


TypeError: All intermediate steps should be transformers and implement fit and transform. 'passthrough' (type <class 'str'>) doesn't

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn.preprocessing import StandardScaler, Normalizer, RobustScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key=''):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

class ItemUnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, keys=[]):
        self.keys = keys

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict.drop(self.keys, axis=1)


pipeline = Pipeline([
   ('union', 
        FeatureUnion(
            transformer_list=[
                ('table', 
                    Pipeline([
                        ('selector1', ItemUnSelector(keys=['opis_cleanv2_nlp', 'opis_clean_nlp'])),
                        ('scaler1', 'passthrough')
                    ])
                ),
                ('description', 
                    Pipeline([
                        ('selector2', ItemSelector()),
                        ('tfidf', TfidfVectorizer()),
                        ('best', TruncatedSVD()),
                        ('scaler2', 'passthrough')
                    ])
                )
            ]
        )    

   ),
   ('regressor', 
        TransformedTargetRegressor()
    )
])

parameters = parameters = {
    'union__transformer_weights': [ { 'table': 3.0, 'description': 1.0}, { 'table': 2.0, 'description': 1.0}, { 'table': 1.0, 'description': 1.0}],

    'union__description__best__n_components': (650, 700, 750),
    'union__description__tfidf__min_df': (3, 4, 5),
    'union__description__tfidf__binary': (True,False),
    'union__description__selector2__key': ['opis_cleanv2_nlp', 'opis_clean_nlp'] ,

    'union__table__scaler1': ['passthrough', StandardScaler(), Normalizer(), RobustScaler()],
    'union__description__scaler2': ['passthrough', StandardScaler(), Normalizer(), RobustScaler(with_centering=False)],

    'regressor': [SVR(kernel='rbf', C=10000), SVR(kernel='linear', C=10000), GradientBoostingRegressor()] ,
}

grid_search = GridSearchCV(pipeline, parameters, verbose=30, cv=2)


with open("data_ML2021_01_07_15_36_52.pkl", "rb") as fh:
  data = pickle.load(fh)
median_imputation(data,'cena_za_metr')
median_imputation(data,'Wielkość (m2)')
testdata = pd.get_dummies(data[:2000],columns=['Na_sprzedaż_przez', 'Rodzaj_nieruchomosci','Liczba_pokoi', 'Liczba_łazienek', 'Parking', 'dzielnica','miasto','poziom_atrakcyjnosci'])
y = testdata['cena_za_metr'].values
X = testdata.drop(['cena_za_metr','cena','mieszkanie_url', 'lokalizacja','opis','opis_clean','opis_cleanv2','data dodania'], axis=1)

t0 = time()
grid_search.fit(X, y)
print("done in %0.3fs" % (time() - t0))

print("Best parameters set:")
print(grid_search.cv_results_)
print(grid_search.best_score_)
print()
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 5184 candidates, totalling 10368 fits
[CV 1/2; 1/5184] START regressor=SVR(C=10000), union__description__best__n_components=650, union__description__scaler2=passthrough, union__description__selector2__key=opis_cleanv2_nlp, union__description__tfidf__binary=True, union__description__tfidf__min_df=3, union__table__scaler1=passthrough, union__transformer_weights={'table': 3.0, 'description': 1.0}
[CV 1/2; 1/5184] END regressor=SVR(C=10000), union__description__best__n_components=650, union__description__scaler2=passthrough, union__description__selector2__key=opis_cleanv2_nlp, union__description__tfidf__binary=True, union__description__tfidf__min_df=3, union__table__scaler1=passthrough, union__transformer_weights={'table': 3.0, 'description': 1.0}; total time=   2.0s
[CV 2/2; 1/5184] START regressor=SVR(C=10000), union__description__best__n_components=650, union__description__scaler2=passthrough, union__description__selector2__key=opis_cleanv2_nlp, union__descri

In [5]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn.preprocessing import StandardScaler, Normalizer, RobustScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key=''):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

class ItemUnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, keys=[]):
        self.keys = keys

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict.drop(self.keys, axis=1)


pipeline = Pipeline([
   ('union', 
        FeatureUnion(
            transformer_list=[
                ('table', 
                    Pipeline([
                        ('selector1', ItemUnSelector(keys=['opis_cleanv2_nlp', 'opis_clean_nlp'])),
                        ('scaler1', 'passthrough')
                    ])
                ),
                ('description', 
                    Pipeline([
                        ('selector2', ItemSelector()),
                        ('tfidf', TfidfVectorizer()),
                        ('best', TruncatedSVD()),
                        ('scaler2', 'passthrough')
                    ])
                )
            ]
        )    

   ),
   ('regressor', 
        TransformedTargetRegressor()
    )
])

parameters = parameters = {
    'union__transformer_weights': [  { 'table': 1.0, 'description': 1.0}],

    'union__description__best__n_components': (700,),
    'union__description__tfidf__min_df': (3,),
    'union__description__tfidf__binary': (True,),
    'union__description__selector2__key': [ 'opis_cleanv2_nlp'] ,
    
    'union__table__scaler1': [ RobustScaler()],
    'union__description__scaler2': [ RobustScaler(with_centering=False)],
    
    'regressor': [ GradientBoostingRegressor()] ,
}

grid_search = GridSearchCV(pipeline, parameters, verbose=20, cv=5)


with open("data_ML2021_01_07_15_36_52.pkl", "rb") as fh:
  data = pickle.load(fh)
median_imputation(data,'cena_za_metr')
median_imputation(data,'Wielkość (m2)')
testdata = pd.get_dummies(data[:2000],columns=['Na_sprzedaż_przez', 'Rodzaj_nieruchomosci','Liczba_pokoi', 'Liczba_łazienek', 'Parking', 'dzielnica','miasto','poziom_atrakcyjnosci'])
y = testdata['cena_za_metr'].values
X = testdata.drop(['cena_za_metr','cena','mieszkanie_url', 'lokalizacja','opis','opis_clean','opis_cleanv2','data dodania'], axis=1)


t0 = time()
grid_search.fit(X, y)
print("done in %0.3fs" % (time() - t0))

print(f'Best score: {grid_search.best_score_}')

print("Best parameters set:")
print()
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START regressor=GradientBoostingRegressor(), union__description__best__n_components=700, union__description__scaler2=RobustScaler(with_centering=False), union__description__selector2__key=opis_cleanv2_nlp, union__description__tfidf__binary=True, union__description__tfidf__min_df=3, union__table__scaler1=RobustScaler(), union__transformer_weights={'table': 1.0, 'description': 1.0}
[CV 1/5; 1/1] END regressor=GradientBoostingRegressor(), union__description__best__n_components=700, union__description__scaler2=RobustScaler(with_centering=False), union__description__selector2__key=opis_cleanv2_nlp, union__description__tfidf__binary=True, union__description__tfidf__min_df=3, union__table__scaler1=RobustScaler(), union__transformer_weights={'table': 1.0, 'description': 1.0}; total time=  24.6s
[CV 2/5; 1/1] START regressor=GradientBoostingRegressor(), union__description__best__n_components=700, union__description__scaler

In [9]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn.preprocessing import StandardScaler, Normalizer, RobustScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key=''):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

class ItemUnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, keys=[]):
        self.keys = keys

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict.drop(self.keys, axis=1)


pipeline = Pipeline([
   ('union', 
        FeatureUnion(
            transformer_list=[
                ('table', 
                    Pipeline([
                        ('selector1', ItemUnSelector(keys=['opis_cleanv2_nlp', 'opis_clean_nlp'])),
                        ('scaler1', 'passthrough')
                    ])
                ),
                ('description', 
                    Pipeline([
                        ('selector2', ItemSelector()),
                        ('tfidf', TfidfVectorizer()),
                        ('best', TruncatedSVD()),
                        ('scaler2', 'passthrough')
                    ])
                )
            ]
        )    

   ),
   ('regressor', 
        TransformedTargetRegressor()
    )
])

parameters = parameters = {
    'union__transformer_weights': [ { 'table': 1.0, 'description': 1.0}],

    'union__description__best__n_components': (650,),
    'union__description__tfidf__min_df': (3, 4),
    'union__description__tfidf__binary': (True,False),
    'union__description__selector2__key': [ 'opis_cleanv2_nlp'] ,
    
    'union__table__scaler1': [ RobustScaler()],
    'union__description__scaler2': [ RobustScaler(with_centering=False)],
    
    'regressor': [ GradientBoostingRegressor()] ,
}

grid_search = GridSearchCV(pipeline, parameters, verbose=20, cv=2)


with open("data_ML2021_01_07_15_36_52.pkl", "rb") as fh:
  data = pickle.load(fh)
median_imputation(data,'cena_za_metr')
median_imputation(data,'Wielkość (m2)')
testdata = pd.get_dummies(data,columns=['Na_sprzedaż_przez', 'Rodzaj_nieruchomosci','Liczba_pokoi', 'Liczba_łazienek', 'Parking', 'dzielnica','miasto','poziom_atrakcyjnosci'])
y = testdata['cena_za_metr'].values
X = testdata.drop(['cena_za_metr','cena','mieszkanie_url', 'lokalizacja','opis','opis_clean','opis_cleanv2','data dodania'], axis=1)


t0 = time()
grid_search.fit(X, y)
print("done in %0.3fs" % (time() - t0))

print(f'Best score: {grid_search.best_score_}')

print("Best parameters set:")
print()
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV 1/2; 1/4] START regressor=GradientBoostingRegressor(), union__description__best__n_components=650, union__description__scaler2=RobustScaler(with_centering=False), union__description__selector2__key=opis_cleanv2_nlp, union__description__tfidf__binary=True, union__description__tfidf__min_df=3, union__table__scaler1=RobustScaler(), union__transformer_weights={'table': 1.0, 'description': 1.0}
[CV 1/2; 1/4] END regressor=GradientBoostingRegressor(), union__description__best__n_components=650, union__description__scaler2=RobustScaler(with_centering=False), union__description__selector2__key=opis_cleanv2_nlp, union__description__tfidf__binary=True, union__description__tfidf__min_df=3, union__table__scaler1=RobustScaler(), union__transformer_weights={'table': 1.0, 'description': 1.0}; total time= 5.2min
[CV 2/2; 1/4] START regressor=GradientBoostingRegressor(), union__description__best__n_components=650, union__description__scaler

In [10]:
print(f'Best score: {grid_search.best_score_}')


Best score: 0.608527733721302
